## Data cleaning and initial EDA
1. What columns do we keep?
2. What is an acceptable null count to get away with? Should we fill in with mean of other columns, or some ridiculous value?
3. Missing demographic data from 3 counties, which should be ok to omit. Can still include in initial chloropleth plots, but not in predictions. 
4. Classes needed? Probably not for cleaning. Functions? Also unlikely for cleaning since this set is already pretty clean. One hot encoder for categorical variables like rural/urban (metro, adjacent to metro, rural)


In [1]:
%load_ext autoreload
%autoreload 2

# Code for cleaning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

# Helper functions
from src import cleaners

In [2]:
data = pd.read_csv('../data/election-context-2018.csv')

In [3]:
# Show all columns
pd.set_option("display.max_columns", 40)
data.head()

,state,county,fips,trump16,clinton16,otherpres16,romney12,obama12,otherpres12,demsen16,repsen16,othersen16,demhouse16,rephouse16,otherhouse16,demgov16,repgov16,othergov16,repgov14,demgov14,othergov14,total_population,cvap,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc
0,Alabama,Autauga,1001,18172,5936,865,17379,6363,190,6331.0,18220.0,62.0,7544.0,14315.0,2258.0,NaN,NaN,NaN,9427.0,3638.0,0.0,55049.0,40690.0,75.683482,18.370906,2.572254,24.316518,1.838362,51.176225,40.037058,13.978456,53099.0,5.591657,12.417046,75.407229,10.002112,74.065601,42.002162,2.0
1,Alabama,Baldwin,1003,72883,18458,3874,66016,18424,898,19145.0,74021.0,248.0,0.0,76995.0,1991.0,NaN,NaN,NaN,37650.0,8416.0,0.0,199510.0,151770.0,83.178788,9.225603,4.366698,16.821212,3.269510,51.194928,35.474412,18.714851,51365.0,6.286843,9.972418,70.452889,7.842227,68.405607,42.279099,3.0
2,Alabama,Barbour,1005,5454,4871,144,5550,5912,47,4777.0,5436.0,16.0,5297.0,4286.0,463.0,NaN,NaN,NaN,3111.0,3651.0,0.0,26614.0,20375.0,45.885624,47.888329,4.309762,54.114376,2.859397,46.498084,37.664387,16.528895,33956.0,12.824738,26.235928,87.132213,19.579752,81.364746,67.789635,6.0
3,Alabama,Bibb,1007,6738,1874,207,6132,2202,86,2082.0,6612.0,17.0,1971.0,6670.0,15.0,NaN,NaN,NaN,3525.0,1368.0,0.0,22572.0,17590.0,74.765196,21.212121,2.223994,25.234804,1.351232,46.464646,37.329435,14.885699,39776.0,7.146827,19.301587,88.000000,15.020490,87.471774,68.352607,1.0
4,Alabama,Blount,1009,22859,2156,573,20757,2970,279,2980.0,22169.0,48.0,2390.0,22367.0,47.0,NaN,NaN,NaN,12074.0,2178.0,0.0,57704.0,42430.0,87.657701,1.557951,8.727298,12.342299,4.271801,50.485235,37.240053,17.192916,46212.0,5.953833,19.968585,86.950243,16.643368,86.163610,89.951502,1.0


In [4]:
len(data.columns)

39

In [141]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3114 entries, 0 to 3113
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3114 non-null   object 
 1   county                  3114 non-null   object 
 2   fips                    3114 non-null   int64  
 3   trump16                 3114 non-null   int64  
 4   clinton16               3114 non-null   int64  
 5   otherpres16             3114 non-null   int64  
 6   romney12                3114 non-null   int64  
 7   obama12                 3114 non-null   int64  
 8   otherpres12             3114 non-null   int64  
 9   demsen16                1942 non-null   float64
 10  repsen16                1942 non-null   float64
 11  othersen16              1942 non-null   float64
 12  demhouse16              2862 non-null   float64
 13  rephouse16              2862 non-null   float64
 14  otherhouse16            2862 non-null   

In [142]:
# Drop columns and rows with na
data = data.drop(columns = data.columns[9:21])
data = data.dropna()
data.head()

,state,county,fips,trump16,clinton16,otherpres16,romney12,obama12,otherpres12,total_population,cvap,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc
0,Alabama,Autauga,1001,18172,5936,865,17379,6363,190,55049.0,40690.0,75.683482,18.370906,2.572254,24.316518,1.838362,51.176225,40.037058,13.978456,53099.0,5.591657,12.417046,75.407229,10.002112,74.065601,42.002162,2.0
1,Alabama,Baldwin,1003,72883,18458,3874,66016,18424,898,199510.0,151770.0,83.178788,9.225603,4.366698,16.821212,3.269510,51.194928,35.474412,18.714851,51365.0,6.286843,9.972418,70.452889,7.842227,68.405607,42.279099,3.0
2,Alabama,Barbour,1005,5454,4871,144,5550,5912,47,26614.0,20375.0,45.885624,47.888329,4.309762,54.114376,2.859397,46.498084,37.664387,16.528895,33956.0,12.824738,26.235928,87.132213,19.579752,81.364746,67.789635,6.0
3,Alabama,Bibb,1007,6738,1874,207,6132,2202,86,22572.0,17590.0,74.765196,21.212121,2.223994,25.234804,1.351232,46.464646,37.329435,14.885699,39776.0,7.146827,19.301587,88.000000,15.020490,87.471774,68.352607,1.0
4,Alabama,Blount,1009,22859,2156,573,20757,2970,279,57704.0,42430.0,87.657701,1.557951,8.727298,12.342299,4.271801,50.485235,37.240053,17.192916,46212.0,5.953833,19.968585,86.950243,16.643368,86.163610,89.951502,1.0


## Let's create new features
1. Get percentage of votes for trump, clinton, other instead of raw numbers
2. Same for romney, obama, other in 2012?
3. Column for percentage of voting age population who voted

In [143]:
cleaners.get_percentage(data)
data.head()

,state,county,fips,trump16,clinton16,otherpres16,romney12,obama12,otherpres12,total_population,cvap,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc,trump16_pct,clinton16_pct,otherpres16_pct,romney12_pct,obama12_pct,otherpres12_pct,cvap_pct12
0,Alabama,Autauga,1001,18172,5936,865,17379,6363,190,55049.0,40690.0,75.683482,18.370906,2.572254,24.316518,1.838362,51.176225,40.037058,13.978456,53099.0,5.591657,12.417046,75.407229,10.002112,74.065601,42.002162,2.0,72.766588,23.769671,3.463741,72.618252,26.587832,0.793916,58.815434
1,Alabama,Baldwin,1003,72883,18458,3874,66016,18424,898,199510.0,151770.0,83.178788,9.225603,4.366698,16.821212,3.269510,51.194928,35.474412,18.714851,51365.0,6.286843,9.972418,70.452889,7.842227,68.405607,42.279099,3.0,76.545712,19.385601,4.068687,77.358269,21.589444,1.052286,56.228504
2,Alabama,Barbour,1005,5454,4871,144,5550,5912,47,26614.0,20375.0,45.885624,47.888329,4.309762,54.114376,2.859397,46.498084,37.664387,16.528895,33956.0,12.824738,26.235928,87.132213,19.579752,81.364746,67.789635,6.0,52.096666,46.527844,1.375490,48.223130,51.368494,0.408376,56.485890
3,Alabama,Bibb,1007,6738,1874,207,6132,2202,86,22572.0,17590.0,74.765196,21.212121,2.223994,25.234804,1.351232,46.464646,37.329435,14.885699,39776.0,7.146827,19.301587,88.000000,15.020490,87.471774,68.352607,1.0,76.403220,21.249575,2.347205,72.826603,26.152019,1.021378,47.868107
4,Alabama,Blount,1009,22859,2156,573,20757,2970,279,57704.0,42430.0,87.657701,1.557951,8.727298,12.342299,4.271801,50.485235,37.240053,17.192916,46212.0,5.953833,19.968585,86.950243,16.643368,86.163610,89.951502,1.0,89.334844,8.425825,2.239331,86.465884,12.371907,1.162209,56.577893


In [144]:
# Let's store population data separately. 
population_data = data[['state', 'county', 'fips', 'total_population']]
population_data.head()

,state,county,fips,total_population
0,Alabama,Autauga,1001,55049.0
1,Alabama,Baldwin,1003,199510.0
2,Alabama,Barbour,1005,26614.0
3,Alabama,Bibb,1007,22572.0
4,Alabama,Blount,1009,57704.0


In [145]:
# Now drop other columns. FIPS won't be used for analysis, but we're going to keep it for now for EDA
columns_to_drop = ['trump16', 'clinton16', 'otherpres16', 'romney12', 'obama12', 'otherpres12',
                  'cvap', 'total_population']

In [146]:
data = data.drop(columns = columns_to_drop)

## Now do OHE on rural columns
But first, figure out which values to keep
### ruralurban\_cc
- **Description**: rural-urban continuum codes
- **Year/s**: 2013
- **Source**: [USDA Economic Research Service](https://www.ers.usda.gov/data-products/rural-urban-continuum-codes/)
- **Coding**:

| Code | Description |
| --- | --- |
| 1 | Counties in metro areas of 1 million population or more |
| 2 | Counties in metro areas of 250,000 to 1 million population |
| 3 | Counties in metro areas of fewer than 250,000 population |
| 4 | Urban population of 20,000 or more, adjacent to a metro area |
| 5 | Urban population of 20,000 or more, not adjacent to a metro area |
| 6 | Urban population of 2,500 to 19,999, adjacent to a metro area |
| 7 | Urban population of 2,500 to 19,999, not adjacent to a metro area |
| 8 | Completely rural or less than 2,500 urban population, adjacent to a metro area |
| 9 | Completely rural or less than 2,500 urban population, adjacent to a metro area |


1. Group all metro areas together

In [147]:
# OHE for rural codes
cleaners.ohe_rural(data)
print(data.iloc[:,25:].sum())
data.head()

cvap_pct12            179932.044193
metro                   1162.000000
rural                    627.000000
urban_metroadj           806.000000
urban_not_metroadj       516.000000
dtype: float64


,state,county,fips,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc,trump16_pct,clinton16_pct,otherpres16_pct,romney12_pct,obama12_pct,otherpres12_pct,cvap_pct12,metro,rural,urban_metroadj,urban_not_metroadj
0,Alabama,Autauga,1001,75.683482,18.370906,2.572254,24.316518,1.838362,51.176225,40.037058,13.978456,53099.0,5.591657,12.417046,75.407229,10.002112,74.065601,42.002162,2.0,72.766588,23.769671,3.463741,72.618252,26.587832,0.793916,58.815434,1,0,0,0
1,Alabama,Baldwin,1003,83.178788,9.225603,4.366698,16.821212,3.269510,51.194928,35.474412,18.714851,51365.0,6.286843,9.972418,70.452889,7.842227,68.405607,42.279099,3.0,76.545712,19.385601,4.068687,77.358269,21.589444,1.052286,56.228504,1,0,0,0
2,Alabama,Barbour,1005,45.885624,47.888329,4.309762,54.114376,2.859397,46.498084,37.664387,16.528895,33956.0,12.824738,26.235928,87.132213,19.579752,81.364746,67.789635,6.0,52.096666,46.527844,1.375490,48.223130,51.368494,0.408376,56.485890,0,0,1,0
3,Alabama,Bibb,1007,74.765196,21.212121,2.223994,25.234804,1.351232,46.464646,37.329435,14.885699,39776.0,7.146827,19.301587,88.000000,15.020490,87.471774,68.352607,1.0,76.403220,21.249575,2.347205,72.826603,26.152019,1.021378,47.868107,1,0,0,0
4,Alabama,Blount,1009,87.657701,1.557951,8.727298,12.342299,4.271801,50.485235,37.240053,17.192916,46212.0,5.953833,19.968585,86.950243,16.643368,86.163610,89.951502,1.0,89.334844,8.425825,2.239331,86.465884,12.371907,1.162209,56.577893,1,0,0,0


In [148]:
data = data.drop(columns=['ruralurban_cc'])

In [149]:
# Save cleaned data and population data
data.to_csv('../data/cleaned_data.csv')
population_data.to_csv('../data/population.csv')

In [150]:
data.describe()

,fips,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,trump16_pct,clinton16_pct,otherpres16_pct,romney12_pct,obama12_pct,otherpres12_pct,cvap_pct12,metro,rural,urban_metroadj,urban_not_metroadj
count,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.00000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000,3111.000000
mean,30646.730633,77.363025,8.962990,8.988876,22.636975,4.617699,49.976043,37.342346,17.634368,47817.63838,7.064097,14.225599,79.216478,11.044488,77.001023,58.481588,63.255408,31.517591,5.227001,59.594595,38.426944,1.978460,57.837366,0.373513,0.201543,0.259081,0.165863
std,14984.498360,19.742492,14.496561,13.656582,19.742492,5.631732,2.333077,5.439730,4.442080,12482.36596,3.228842,6.543414,9.138770,5.327779,10.358919,31.450943,15.688546,15.234159,3.259927,14.843237,14.714982,1.396700,9.637659,0.483814,0.401217,0.438200,0.372017
min,1001.000000,0.760221,0.000000,0.000000,0.000000,0.000000,21.513413,11.842105,3.854570,18972.00000,0.000000,1.278772,19.789880,0.000000,9.188199,0.000000,4.087474,0.000000,0.334448,7.193430,3.448276,0.000000,16.999430,0.000000,0.000000,0.000000,0.000000
25%,19038.000000,65.581363,0.587323,1.938639,6.735279,1.305247,49.487857,34.163145,14.831727,39681.00000,4.952997,9.307582,75.317416,7.231642,72.886802,33.233871,54.439540,20.347762,3.176054,50.293967,27.772945,1.220976,51.178934,0.000000,0.000000,0.000000,0.000000
50%,29207.000000,84.712353,2.121106,3.832296,15.287647,2.653356,50.428902,37.000969,17.280145,46250.00000,6.761641,12.773185,81.474694,9.938900,79.452055,59.412038,66.337972,28.275769,4.765565,60.781381,37.128713,1.733233,57.754218,0.000000,0.000000,0.000000,0.000000
75%,46004.000000,93.264721,10.168007,9.164585,34.418637,5.560713,51.132910,39.984573,19.933773,53563.50000,8.774941,18.307272,85.567853,13.947282,83.918746,87.037801,74.903635,39.888543,6.603790,70.280499,47.445567,2.433085,64.214213,1.000000,0.000000,1.000000,0.000000
max,56045.000000,100.000000,86.184910,98.958862,99.239779,52.229868,58.499127,70.981316,53.106375,125672.00000,29.927415,51.479358,97.014925,41.758287,95.918367,100.000000,96.033403,90.863822,44.223382,95.862069,90.913114,12.163422,127.872340,1.000000,1.000000,1.000000,1.000000
